### Part I - Reddit Data Collection


In [ ]:
### import praw
import datetime
import csv
import praw
import os
from pathlib import Path

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

#subreddit name list
sub_name = ['india']

#number of post to retrieve
post_num = 250

#OAuth credentials from reddit account
reddit = praw.Reddit(
                     client_id='hDIJdjc2k5VzTA',
                     client_secret='fT9JnoUGAUAqDipudbAZ1XgeLFM',
                     username='HelloWorld13211',
                     password='Qwerty@123',
                     user_agent='web scrapper'
                     )

#writing retrieved list of posts into csv
def write_csv(to_csv, csv_name):
    try:
        with open(csv_name, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerows(to_csv)
            file.close()
    except Exception as e:
        print(e)

#retrieve post from reddit ~ duh
def retrieve_post(sub,post_number,csv_name):
    subreddit = reddit.subreddit(sub)
    sub_hot = subreddit.hot(limit=post_number)
    to_csv = [['flair_name','title']]
    
    print('Scrapping {}...'.format(sub))
    try:
        for post in sub_hot:
            if not post.stickied:
                retrieved = [
                            post.link_flair_text,
                            post.title,
                            ]
                
                to_csv.append(retrieved)
               
        write_csv(to_csv, csv_name)
    except Exception as e:
        print(e)

for sub in sub_name:
    #Note: The csv file names are saved in {subreddit name}_yyyymmdd.csv format
    csv_name = '{}.csv'.format(sub)
    #csv path
    csv_file = Path(csv_name)

    #check if csv file already exist to prevent overwriting before retrieving post from reddit
    if csv_file.exists():
        print('File {} already exist!'.format(csv_name))
    else:
        retrieve_post(sub,post_num,csv_name)
    print("done")


### Reading of CSV File & print the following


In [ ]:
import numpy as np
import pandas as pd
df=pd.read_csv("india.csv ")
df.flair_name=pd.Categorical(df.flair_name)
df['flair']=df.flair_name.cat.codes
df

### Part II- Exploratory Data Analysis
This analysis is done on terms of graph.

The first graph describes the relation b/w each Flair_name and the no of posts.
this means that how many Post of a particular Flair_name exists. 

In [ ]:
import matplotlib.pyplot as plt
flair_count = pd.value_counts(df.flair_name)
flair_count.plot(kind="bar")
flair_count

This graph shows the occurance of the different word in the post_title.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
data_x=df.title
data_y=df.flair
cv=TfidfVectorizer(min_df=1)
X=cv.fit_transform(data_x)
Y=data_y.astype(int)
plt.spy(X)

### Part III - Flare Detector


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=4)
mnb=MultinomialNB()
mnb.fit(x_train,y_train)
title=input("Enter the title : ")
title=cv.transform([title])
index=mnb.predict(title)
index=int(index)
result=df.flair_name.cat.categories[index]

### Performance of Classifier


1)How well it works?
:-The flare detector works very well and accurate.


2)What is it good at?
:-It is accurate in classifying the posts from the r/india through csv file.


3)What is it bad at?
:-When there is very huge imbalance dataset the classifier works but it is not so accurate. 